<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/Correspondences_By_Interpolation_Endonasal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONFIG

In [ ]:
import config

In [ ]:
config.DATASET_PATH

'dataset/endo_videos'

In [24]:

# https://drive.google.com/file/d/1q-952htGhoEDVlYmnqkAA1UheCg-Yalg/view?usp=sharing
# https://drive.google.com/file/d/1sV4j-wGBpfvL0O-VtTcyN6nygNjhaeuQ/view?usp=sharing
import gdown
url = 'https://drive.google.com/uc?id=1q-952htGhoEDVlYmnqkAA1UheCg-Yalg'
gdown.download(url,'10.mp4',quiet=True) 

url = 'https://drive.google.com/uc?id=1sV4j-wGBpfvL0O-VtTcyN6nygNjhaeuQ'
gdown.download(url,'09.mp4',quiet=True) 

'09.mp4'

In [25]:
import cv2
import os
 
# Function to extract frames
def FrameCapture(video_path=None, save_path=None): 
    # Path to video file
    vidObj = cv2.VideoCapture(video_path) 
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(video_path, 'original fps:',fps)
    # Used as counter variable
    count = 0
    # checks whether frames were extracted
    success = 1 
    while success:
        # vidObj object calls read
        # function extract frames
        success, image = vidObj.read()
        # Saves the frames with frame-count
        try:
            cv2.imwrite(save_path+"/frame%06d.jpg" % count, image)
        except:
            print('Done!')
            continue
        count += 1

if not os.path.isdir("endonasal"):
    os.makedirs('endonasal/10')
    os.makedirs('endonasal/09')


FrameCapture(video_path='10.mp4', save_path='endonasal/10')
print('total frames:')
!ls endonasal | wc -l

FrameCapture(video_path='09.mp4', save_path='endonasal/09')

10.mp4 original fps: 25.025126033800067
Done!
total frames:
2
09.mp4 original fps: 25.025126033800067
Done!
total frames:
2


In [27]:
print('total frames in endonasal/10:')
!ls endonasal/10 | wc -l

print('total frames in endonasal/09:')
!ls endonasal/09 | wc -l

total frames in endonasal/10:
1984
total frames in endonasal/09:
1041


In [29]:
import glob
sequence_path = '09'
img_paths = sorted(glob.glob(f'{sequence_path}/*.jpg'))
img_paths

[]

In [ ]:
import PIL.Image
from torch.utils.data import Dataset
import cv2
from torch import concat
import PIL
#from utils import config
from torchvision import transforms
import glob
import numpy as np

class endonasal(Dataset):
    def __init__(self, sequence_paths=None, transform=None):
        self.transform = transform
        self.training_triplet_paths = []
        # store the image and mask filepaths, and augmentation
        # transforms
        for sequence_path in sequence_paths:
            # in each sequence there is many folders. We are interested in the image_02
            # and image_03 which are the RGB. They represent stereo images so we simply take
            # each as a new training example.
            img_paths = sorted(glob.glob(f'{sequence_path}/*.jpg'))
            # print(f'found {len(img_paths)} image paths for {sequence_path}/{mono_folder}')
            for idx in range(len(img_paths)-2):
                image_path_1 = img_paths[idx]
                label_path = img_paths[idx + 1]  # middle image acts as interpolated version of images
                image_path_2 = img_paths[idx + 2]
                self.training_triplet_paths.append([image_path_1, label_path, image_path_2])

    def __len__(self):
        # return the number of total samples contained in the dataset
        # print(f'found {len(self.training_triplet_paths)} examples')
        return int(len(self.training_triplet_paths))

    def __getitem__(self, idx):
        # grab the triplet of training data:
        image_path_1 = self.training_triplet_paths[idx][0]
        label_path = self.training_triplet_paths[idx][1]  # middle image acts as interpolated version of images
        image_path_2 = self.training_triplet_paths[idx][2]

        # load the 3 images from disk, swap its channels from BGR to RGB,
        image1 = PIL.Image.open(image_path_1).convert('RGB')
        image2 = PIL.Image.open(image_path_2).convert('RGB')
        label = PIL.Image.open(label_path).convert('RGB')

        # check to see if we are applying any transformations (eg. resize, convert to tensor etc
        if self.transform:
            image1, image2, label = self.transform(image1), self.transform(image2), self.transform(label)

        # concat first and 3rd images to create input. Output is the middle img (label)
        input = concat([image1, image2])
        return input, label

root = 'endonasal'
sequences_train = ['09', '10']

transform_all = transforms.Compose([
    transforms.Resize((128, 384)),
    transforms.ToTensor(),
])

train_dataset = endonasal(root=root, sequences=sequences_train, transform=transform_all)


In [ ]:
import PIL.Image
from torch.utils.data import Dataset
import cv2
from torch import concat
import PIL
#from utils import config
from torchvision import transforms
import glob
import numpy as np

class KITTI(Dataset):
    def __init__(self, sequence_paths=None, transform=None):
        self.transform = transform
        self.training_triplet_paths = []
        # store the image and mask filepaths, and augmentation
        # transforms
        for sequence_path in sequence_paths:
            # in each sequence there is many folders. We are interested in the image_02
            # and image_03 which are the RGB. They represent stereo images so we simply take
            # each as a new training example.
            for mono_folder in ['image_02', 'image_03']:
                img_paths = sorted(glob.glob(f'{sequence_path}/{mono_folder}/data/*.png'))
                # print(f'found {len(img_paths)} image paths for {sequence_path}/{mono_folder}')
                for idx in range(len(img_paths)-2):
                    image_path_1 = img_paths[idx]
                    label_path = img_paths[idx + 1]  # middle image acts as interpolated version of images
                    image_path_2 = img_paths[idx + 2]
                    self.training_triplet_paths.append([image_path_1, label_path, image_path_2])

    def __len__(self):
        # return the number of total samples contained in the dataset
        # print(f'found {len(self.training_triplet_paths)} examples')
        return int(len(self.training_triplet_paths))

    def __getitem__(self, idx):
        # grab the triplet of training data:
        image_path_1 = self.training_triplet_paths[idx][0]
        label_path = self.training_triplet_paths[idx][1]  # middle image acts as interpolated version of images
        image_path_2 = self.training_triplet_paths[idx][2]

        # load the 3 images from disk, swap its channels from BGR to RGB,
        image1 = PIL.Image.open(image_path_1).convert('RGB')
        image2 = PIL.Image.open(image_path_2).convert('RGB')
        label = PIL.Image.open(label_path).convert('RGB')

        # check to see if we are applying any transformations (eg. resize, convert to tensor etc
        if self.transform:
            image1, image2, label = self.transform(image1), self.transform(image2), self.transform(label)

        # concat first and 3rd images to create input. Output is the middle img (label)
        input = concat([image1, image2])
        return input, label

class ENDO(Dataset):
    def __init__(self, sequence_paths=None, transform=None):
        self.transform = transform
        self.training_triplet_paths = []
        # store the image and mask filepaths, and augmentation
        # transforms
        for sequence_path in sequence_paths:
            #for mono_folder in ['image_02', 'image_03']:
            img_paths = sorted(glob.glob(f'{sequence_path}/*.*'))
            print(f'found {len(img_paths)} image paths for {sequence_path}')
            for idx in range(len(img_paths)-300): # 2
                image_path_1 = img_paths[idx]
                label_path = img_paths[idx + 150] # 1 # middle image acts as interpolated version of images
                image_path_2 = img_paths[idx + 300] # 2
                self.training_triplet_paths.append([image_path_1, label_path, image_path_2])

    def __len__(self):
        # return the number of total samples contained in the dataset
        # print(f'found {len(self.training_triplet_paths)} examples')
        return int(len(self.training_triplet_paths))

    def __getitem__(self, idx):
        # grab the triplet of training data:
        image_path_1 = self.training_triplet_paths[idx][0]
        label_path = self.training_triplet_paths[idx][1]  # middle image acts as interpolated version of images
        image_path_2 = self.training_triplet_paths[idx][2]

        # load the 3 images from disk, swap its channels from BGR to RGB,
        image1 = PIL.Image.open(image_path_1).convert('RGB')
        image2 = PIL.Image.open(image_path_2).convert('RGB')
        label = PIL.Image.open(label_path).convert('RGB')

        # check to see if we are applying any transformations (eg. resize, convert to tensor etc
        if self.transform:
            image1, image2, label = self.transform(image1), self.transform(image2), self.transform(label)

        # concat first and 3rd images to create input. Output is the middle img (label)
        input = concat([image1, image2])
        return input, label




class ENDO_VIDEO(Dataset):
    def __init__(self, video_paths=None, transform=None):
        self.transform = transform
        self.training_triplet_paths = []
        # store the image and mask filepaths, and augmentation
        # transforms

        skip_factor = 1

        for video_path in video_paths:
            vid_name = video_path.split('/')[-1]

            # count num frames in vid
            video = cv2.VideoCapture(video_path)
            num_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
            print(f'found {num_of_frames} frames for video {vid_name}')

            
            images_1 = np.arange(0,num_of_frames-2)
            images_2 = np.arange(1,num_of_frames-1)
            images_3 = np.arange(2,num_of_frames)
            vid_path_name = [video_path]*(num_of_frames-2)

            self.training_triplet_paths += list(zip(vid_path_name,images_1, images_2, images_3))

        video.release()  

    def __len__(self):
        # return the number of total samples contained in the dataset
        # print(f'found {len(self.training_triplet_paths)} examples')
        return int(len(self.training_triplet_paths))

    def __getitem__(self, idx):
        
        print(f'index {idx}')
        # grab the triplet of training data:
        vid_path_name = self.training_triplet_paths[idx][0]

        cap = cv2.VideoCapture(vid_path_name)

        image_frame_1 = self.training_triplet_paths[idx][1]
        label_frame = self.training_triplet_paths[idx][2]  # middle image acts as interpolated version of images
        image_frame_2 = self.training_triplet_paths[idx][3]

        # load the 3 images from disk, swap its channels from BGR to RGB,
        image1 = cap.set(cv2.CAP_PROP_POS_FRAMES, image_frame_1) # .convert('RGB')
        res, image1 = cap.read()
        image2 = cap.set(cv2.CAP_PROP_POS_FRAMES, image_frame_2)
        res, image2 = cap.read()
        label = cap.set(cv2.CAP_PROP_POS_FRAMES, label_frame) # .COLOR_BGR2RGB
        res, label = cap.read()

        # convert to PIL
        image1 = PIL.Image.fromarray(image1).convert('RGB')
        image2 = PIL.Image.fromarray(image2).convert('RGB')
        label = PIL.Image.fromarray(label).convert('RGB')

        # check to see if we are applying any transformations (eg. resize, convert to tensor etc
        if self.transform:
            image1, image2, label = self.transform(image1), self.transform(image2), self.transform(label)

        # concat first and 3rd images to create input. Output is the middle img (label)
        input = concat([image1, image2])

        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()
        return input, label

# MODEL

In [ ]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:

""" Full assembly of the parts to form the complete network """

#from utils.unet_parts import *


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64) # 6 channels ("stack inputs")
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes) # 3 classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

# TRAIN

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from utils import config
import glob
import os
from urllib import request, error
import zipfile
import shutil
import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW
import torch

#from utils import config
#from utils.dataset import KITTI, ENDO, ENDO_VIDEO
#from utils.model import UNet
from torchmetrics import StructuralSimilarityIndexMeasure


def seed_everything(seed=42):
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


class SSIM_Loss_Lib(nn.Module):
    def __init__(self, data_range=1):
        super().__init__()
        self.ssim = StructuralSimilarityIndexMeasure(data_range=data_range)

    def forward(self, img1, img2):
        return 1 - self.ssim(img1, img2)


def download_data(file):
    print(f'downloading {file}...')
    try:
        URL = f'https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/{file}/{file}_sync.zip'
        request.urlretrieve(URL, f'dataset/{file}.zip')
        print('download complete')
    except error.HTTPError:
        print(f'could not find file {file}')

    return


def unzip_data(file):
    try:
        print(f'unzipping {file}...')
        with zipfile.ZipFile(f'dataset/{file}.zip', 'r') as zip_ref:
            zip_ref.extractall('dataset')
        os.remove(f'dataset/{file}.zip')
        print('file unzipped')
    except FileNotFoundError:
        print(f'file {file} not found')


def cleaning_data_dirs(new_data_dir):
    # dirs_02 = glob.glob('dataset/*_*_*/*/image_02/data')
    # dirs_03 = glob.glob('dataset/*_*_*/*/image_03/data')
    # dirs = glob.glob('dataset/*_*_*/*/image_0[2-3]/data')

    dirs = glob.glob('dataset/*_*_*/*')
    for source_file in dirs:
        shutil.move(source_file, new_data_dir)

    # remove empty year dirs we've just moved all the files out of
    main_dirs = glob.glob('dataset/*_*_*')
    for dir in main_dirs:
        os.rmdir(f"{dir}")
    return


def train(unet, train_loader, loss_function, optimizer):
    # set the model in training mode
    unet.train()

    # initialize the total training loss
    # total_train_loss = 0

    ssim = StructuralSimilarityIndexMeasure(data_range=1)

    train_loss = []
    ssim_all = []

    # loop over the training set
    # we iterate over our trainLoader dataloader, which provides a batch of samples at a time
    for (i_batch, (input_imgs, label)) in enumerate(train_loader):
        # send the input to the device we are training our model on
        (input_imgs, label) = (input_imgs.to(config.DEVICE), label.to(config.DEVICE))

        # pass input images through unet to get prediction of interpolation
        prediction = unet(input_imgs)

        # compute loss between model prediction and ground truth label
        # print(f'label min: {torch.min(label)}, max of label: {torch.max(label)}')
        # print(f'label min: {torch.min(prediction)}, max of label: {torch.max(prediction)}')
        loss = loss_function(prediction, label)

        # update the parameters of model
        optimizer.zero_grad()  # getting rid of previously accumulated gradients from previous steps
        loss.backward()  # backpropagation
        optimizer.step()  # update model params

        # add the loss to the total training loss so far
        #total_train_loss += loss.item()
        train_loss.append(loss.item())
        ssim_all.append(ssim(prediction.detach().cpu(), label.detach().cpu()).item())

    return np.mean(train_loss), np.mean(ssim_all)


def test(unet, test_loader, loss_function):
    # set the model in evaluation mode
    unet.eval()
    ssim = StructuralSimilarityIndexMeasure(data_range=1)

    test_loss = []
    ssim_all = []

    # switch off gradient computation (as during testing we don't want to get weights.
    with torch.no_grad():
        # loop over the validation set
        for (input_imgs, label) in test_loader:
            # send the input and label to the device
            (test_input_imgs, label) = (input_imgs.to(config.DEVICE), label.to(config.DEVICE))

            # make the predictions and calculate the validation loss
            prediction = unet(test_input_imgs)
            loss = loss_function(prediction, label)
            test_loss.append(loss.item())
            ssim_all.append(ssim(prediction.detach().cpu(), label.detach().cpu()).item())

    return np.mean(test_loss), np.mean(ssim_all)


def get_kitti_data():
    new_data_dir = 'dataset/kitti_raw'
    if not os.path.exists(new_data_dir):
        print('creating dir...')
        # make directory where data will be (inside dataset)
        os.mkdir('dataset')
        os.mkdir(new_data_dir)

    if len(os.listdir(new_data_dir)) == 0:
        # loop through each filename to download and unzip data
        file_names = open('filenames.txt', 'r')
        for file in file_names.readlines():
            download_data(file[:-1])
            unzip_data(file[:-1])
            print(file)

        # move all files to corresponding folder
        cleaning_data_dirs(new_data_dir)

def main():
    print('seeding...')
    seed_everything()

    print('downloading data...')

    load_weights = True

    # transforms that need to be done on the data when retrieved from the disk as PIL Image
    transform_all = transforms.Compose([
        transforms.Resize((128, 384)),
        transforms.ToTensor(),
    ])

    # loading train and test dataset
    dataset_paths = glob.glob(f'{config.DATASET_PATH}/*_*/*')
    
    # splitting to test and train sequences
    if len(dataset_paths) == 1:
        print('only one vid found')
        training_sequence_paths = dataset_paths
        testing_sequence_paths = dataset_paths
    else:
        split_loc = int(len(dataset_paths) / 2)  # finding location where to split train and test
        training_sequence_paths = dataset_paths[:split_loc]
        testing_sequence_paths = dataset_paths[split_loc:]

    # -------- DATA PREP
    if config.data == 'kitti_raw':
        # --- DATA DOWNLOAD
        get_kitti_data()

        # loading paths
        dataset_train = KITTI(sequence_paths=training_sequence_paths, transform=transform_all)
        dataset_test = KITTI(sequence_paths=testing_sequence_paths, transform=transform_all)

    elif config.data == 'endo_data':
        
        dataset_train = ENDO(sequence_paths=training_sequence_paths, transform=transform_all)
        dataset_test = ENDO(sequence_paths=testing_sequence_paths, transform=transform_all)
        
    elif config.data == 'endo_videos':
        dataset_train = ENDO_VIDEO(video_paths=training_sequence_paths, transform=transform_all)
        dataset_test = ENDO_VIDEO(video_paths=testing_sequence_paths, transform=transform_all)

    print(f"[INFO] found {len(dataset_train)} examples in the training set...")
    print(f"[INFO] found {len(dataset_test)} examples in the test set...")

    # create the training and test data loaders

    # We make shuffle = True since we want samples from all classes to be uniformly present in a
    # batch which is important for optimal learning and convergence of batch
    # gradient-based optimization approaches.
    train_loader = DataLoader(dataset_train, shuffle=True,
                              batch_size=config.BATCH_SIZE,
                              pin_memory=config.PIN_MEMORY,
                              num_workers=os.cpu_count())
    test_loader = DataLoader(dataset_test, shuffle=False,
                             batch_size=config.BATCH_SIZE,
                             pin_memory=config.PIN_MEMORY,
                             num_workers=os.cpu_count())

    # initialize our UNet model:
    # # n_channels=3 for RGB images
    # n_classes is the number of probabilities you want to get per pixel
    unet = UNet(n_channels=6, n_classes=3, bilinear=False).to(config.DEVICE)
    # print(unet)

    if load_weights:
        # weights
        checkpoint = torch.load(f'{config.BASE_OUTPUT}/unet_interpolation.pth.tar', map_location=torch.device(config.DEVICE))
        # load weights to model
        unet.load_state_dict(checkpoint['state_dict'])

    # choosing which loss to train UNET with (set this in config file)
    if config.LOSS == 'mse':
        criterion = nn.MSELoss()
    elif config.LOSS == 'l1':
        criterion = nn.L1Loss()
    elif config.LOSS == 'bce':
        criterion = nn.BCEWithLogitsLoss()

    # initialize loss function and optimizer
    loss_function = criterion.to(config.DEVICE)
    opt = AdamW(unet.parameters(), lr=config.INIT_LR)

    # ---------- TRAINING
    # calculate steps per epoch for training and test set
    num_steps_train = len(dataset_train) // config.BATCH_SIZE
    num_steps_test = len(dataset_test) // config.BATCH_SIZE

    # initialize a dictionary to store training history
    train_history = {"train_loss": [], "test_loss": [], "train_ssim":[], 'test_ssim':[]}

    print(f'[INFO] training the network...')

    start_time = time.time()
    best_epoch, best_loss = 0, np.inf
    for epoch in tqdm(range(config.NUM_EPOCHS)):

        # training unet on data
        # total_train_loss = train(unet, train_loader, loss_function, opt)
        avg_train_loss, avg_train_ssim = train(unet, train_loader, loss_function, opt)
        # Once we have processed our entire training set,
        # evaluate our model on the test set to monitor test loss and
        # ensure that our model is not overfitting to the training set.
        # total_test_loss = test(unet, test_loader, loss_function)
        avg_test_loss, avg_test_ssim = test(unet, test_loader, loss_function)
        # calculate the average training and validation loss
        #avg_train_loss = total_train_loss / num_steps_train
        #avg_test_loss = total_test_loss / num_steps_test

        # update our training history
        # train_history["train_loss"].append(avg_train_loss.cpu().detach().numpy())
        # train_history["test_loss"].append(avg_test_loss.cpu().detach().numpy())
        train_history["train_loss"].append(avg_train_loss)
        train_history["test_loss"].append(avg_test_loss)

        train_history["train_ssim"].append(avg_train_ssim)
        train_history["test_ssim"].append(avg_test_ssim)

        # training indo
        print(f'[INFO] EPOCH: {epoch + 1}/{config.NUM_EPOCHS}')
        print("Train loss: {:.6f}, Test loss: {:.4f}".format(
            avg_train_loss, avg_test_loss))
        print("Train ssim: {:.6f}, Test ssim: {:.4f}".format(
            avg_train_ssim, avg_test_ssim))

        # saving model
        if avg_test_loss < best_loss:
            print('saving model...')

            checkpoint = {}

            best_loss = avg_test_loss
            checkpoint['best_epoch'] = epoch

            checkpoint['best_train_loss'] = avg_train_loss
            checkpoint['best_train_ssim'] = avg_train_ssim

            checkpoint['best_test_loss'] = avg_test_loss
            checkpoint['best_test_ssim'] = avg_test_ssim

            checkpoint['state_dict'] = unet.state_dict()
            # torch.save(unet.state_dict(), config.MODEL_PATH)
            torch.save(checkpoint, config.MODEL_PATH)

    # display the total time needed to perform the training
    end_time = time.time()
    print("[INFO] total time taken to train the model: {:.2f}s".format(
        end_time - start_time))

    # plot the training loss
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(train_history["train_loss"], label="train_loss")
    plt.plot(train_history["test_loss"], label="test_loss")
    plt.title("Training Loss on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")
    plt.savefig(config.PLOT_PATH)


if __name__ == '__main__':
    main()


seeding...
downloading data...


UnboundLocalError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')